# Trabajo de Aprendizaje No Supervisado


## Punto 2

- Buscar un **dataset sin etiquetas** que cumpla las siguientes condiciones:
  - Número de variables (**n**) mayor a **10**  
  - Número de ejemplos (**m**) mayor a **10.000**
- Los datasets que incluyan **contenido gráfico, de audio, texto o datos con efectos reales en cualquier ámbito** tendrán **mayor puntuación**.  
- Los **datasets no deben ser similares**, por lo que se debe **coordinar con el universitario Romero Morales Jhojan Erick**.
- Aplicar las siguientes técnicas:
  - **Aprendizaje semisupervisado**
  - **Aprendizaje activo**

In [92]:
# Punto 2: Aprendizaje Semisupervisado y Aprendizaje Activo
# Dataset: ISOLET - Reconocimiento de letras del abecedario por características de audio

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Cargar el dataset ISOLET
isolet_data = pd.read_csv("../Dataset/ISOLET (2).csv")
print(f"Dataset cargado: {isolet_data.shape}")

# Separar características (X) y etiquetas (y)
X = isolet_data.iloc[:, :-1].values  
y = isolet_data.iloc[:, -1].values   

# Convertir etiquetas de string a números si es necesario
if isinstance(y[0], str):
    unique_labels = np.unique(y)
    label_map = {label: i for i, label in enumerate(unique_labels)}
    y = np.array([label_map[label] for label in y])

print(f"Características (X): {X.shape}")
print(f"Etiquetas (y): {y.shape}")
print(f"Número de variables: {X.shape[1]}")
print(f"Número de ejemplos: {X.shape[0]}")
print(f"Número de clases únicas: {len(np.unique(y))}")
print(f"Rango de etiquetas: {min(y)} - {max(y)}")

Dataset cargado: (7796, 618)
Características (X): (7796, 617)
Etiquetas (y): (7796,)
Número de variables: 617
Número de ejemplos: 7796
Número de clases únicas: 26
Rango de etiquetas: 0 - 25


In [93]:
# Preparar el dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

print(f"Entrenamiento - X: {X_train.shape}, y: {y_train.shape}")
print(f"Prueba - X: {X_test.shape}, y: {y_test.shape}")

# Mostrar distribución de clases
unique_labels, counts = np.unique(y_train, return_counts=True)
print(f"\nDistribución de letras en entrenamiento:")
for label, count in zip(unique_labels, counts):
    letter = chr(ord('A') + label) if label <= 25 else f"Clase_{label}"
    print(f"  {letter} (clase {label}): {count} ejemplos")

Entrenamiento - X: (5847, 617), y: (5847,)
Prueba - X: (1949, 617), y: (1949,)

Distribución de letras en entrenamiento:
  A (clase 0): 229 ejemplos
  B (clase 1): 225 ejemplos
  C (clase 2): 227 ejemplos
  D (clase 3): 206 ejemplos
  E (clase 4): 229 ejemplos
  F (clase 5): 231 ejemplos
  G (clase 6): 205 ejemplos
  H (clase 7): 229 ejemplos
  I (clase 8): 223 ejemplos
  J (clase 9): 236 ejemplos
  K (clase 10): 230 ejemplos
  L (clase 11): 225 ejemplos
  M (clase 12): 219 ejemplos
  N (clase 13): 231 ejemplos
  O (clase 14): 241 ejemplos
  P (clase 15): 227 ejemplos
  Q (clase 16): 223 ejemplos
  R (clase 17): 237 ejemplos
  S (clase 18): 216 ejemplos
  T (clase 19): 232 ejemplos
  U (clase 20): 216 ejemplos
  V (clase 21): 217 ejemplos
  W (clase 22): 215 ejemplos
  X (clase 23): 232 ejemplos
  Y (clase 24): 222 ejemplos
  Z (clase 25): 224 ejemplos


## Aprendizaje Semisupervisado

Aplicaremos K-Means para seleccionar muestras representativas y luego entrenar un modelo con pocas etiquetas.

In [94]:
# Aplicar K-Means para encontrar muestras representativas
k = len(np.unique(y_train))  
print(f"Número de clusters K-Means: {k}")

# Aplicar K-Means
kmeans = KMeans(n_clusters=k, random_state=42)
X_audio_dist = kmeans.fit_transform(X_train)

print(f"Shape de distancias: {X_audio_dist.shape}")

Número de clusters K-Means: 26
Shape de distancias: (5847, 26)
Shape de distancias: (5847, 26)


In [95]:
# Seleccionar muestras representativas
idxs = np.argmin(X_audio_dist, axis=0)  
X_representative_audio = X_train[idxs]  

print(f"Shape de muestras representativas: {X_representative_audio.shape}")
print(f"Se seleccionaron {len(idxs)} muestras representativas")

# Obtener las etiquetas reales de las muestras representativas
y_representative_audio = y_train[idxs]

print("\nMuestras representativas seleccionadas:")
for i, (idx, label) in enumerate(zip(idxs, y_representative_audio)):
    letter = chr(ord('A') + label) if label <= 25 else f"Clase_{label}"
    print(f"  Cluster {i+1}: Muestra {idx} → Letra {letter} (clase {label})")

Shape de muestras representativas: (26, 617)
Se seleccionaron 26 muestras representativas

Muestras representativas seleccionadas:
  Cluster 1: Muestra 5303 → Letra Y (clase 24)
  Cluster 2: Muestra 1060 → Letra U (clase 20)
  Cluster 3: Muestra 2387 → Letra Z (clase 25)
  Cluster 4: Muestra 3327 → Letra R (clase 17)
  Cluster 5: Muestra 5796 → Letra W (clase 22)
  Cluster 6: Muestra 536 → Letra P (clase 15)
  Cluster 7: Muestra 4495 → Letra C (clase 2)
  Cluster 8: Muestra 216 → Letra T (clase 19)
  Cluster 9: Muestra 1956 → Letra U (clase 20)
  Cluster 10: Muestra 1254 → Letra J (clase 9)
  Cluster 11: Muestra 2365 → Letra E (clase 4)
  Cluster 12: Muestra 297 → Letra N (clase 13)
  Cluster 13: Muestra 3645 → Letra I (clase 8)
  Cluster 14: Muestra 3900 → Letra F (clase 5)
  Cluster 15: Muestra 4697 → Letra X (clase 23)
  Cluster 16: Muestra 2348 → Letra T (clase 19)
  Cluster 17: Muestra 1889 → Letra W (clase 22)
  Cluster 18: Muestra 439 → Letra B (clase 1)
  Cluster 19: Muestra 16

In [96]:
# Entrenar modelo con muestras representativas
log_reg_representative = LogisticRegression(multi_class="ovr", solver="lbfgs", max_iter=5000, random_state=42)
log_reg_representative.fit(X_representative_audio, y_representative_audio)

# Evaluar el modelo en el conjunto de prueba
accuracy_representative = log_reg_representative.score(X_test, y_test)
print(f"Precisión usando solo {k} muestras representativas: {accuracy_representative:.4f} = {accuracy_representative * 100:.2f}%")

Precisión usando solo 26 muestras representativas: 0.5162 = 51.62%


In [97]:
# Comparación con modelo aleatorio
n_representative = len(X_representative_audio)
log_reg_random = LogisticRegression(multi_class="ovr", solver="lbfgs", max_iter=5000, random_state=42)
log_reg_random.fit(X_train[:n_representative], y_train[:n_representative])

# Evaluar modelo aleatorio
accuracy_random = log_reg_random.score(X_test, y_test)
print(f"Precisión usando {n_representative} muestras aleatorias: {accuracy_random:.2f} = {accuracy_random * 100:.2f}%")

# Mostrar la diferencia
improvement = accuracy_representative - accuracy_random
print(f"Mejora del aprendizaje semisupervisado: {improvement:+.2f} = {improvement * 100:+.2f}%")


Precisión usando 26 muestras aleatorias: 0.40 = 39.82%
Mejora del aprendizaje semisupervisado: +0.12 = +11.80%


In [98]:
# Propagación de etiquetas
y_train_propagated = np.empty(len(X_train))
for i in range(k):
    y_train_propagated[kmeans.labels_ == i] = y_representative_audio[i]

print(f"Etiquetas propagadas a {len(y_train_propagated)} muestras")

# Entrenar con las etiquetas propagadas
log_reg_propagated = LogisticRegression(multi_class="ovr", solver="lbfgs", max_iter=5000, random_state=42)
n_semisupervised = 1000
log_reg_propagated.fit(X_train[:n_semisupervised], y_train_propagated[:n_semisupervised])

# Evaluar modelo con propagación
accuracy_propagated = log_reg_propagated.score(X_test, y_test)
print(f"Precisión con {n_semisupervised} muestras semi-etiquetadas: {accuracy_propagated:.4f}")

Etiquetas propagadas a 5847 muestras
Precisión con 1000 muestras semi-etiquetadas: 0.5428


## Aprendizaje Activo

Identificaremos las muestras más inciertas para el modelo y las usaremos para mejorar el entrenamiento.

In [99]:
# Calcular probabilidades de predicción para aprendizaje activo
n_active_samples = 1000
probas = log_reg_propagated.predict_proba(X_train[:n_active_samples])

# Obtener el índice de la clase más probable para cada muestra
labels_ixs = np.argmax(probas, axis=1)

# Obtener la confianza (probabilidad máxima) de cada predicción
confidences = probas[np.arange(len(probas)), labels_ixs]

print(f"Probabilidades calculadas para {n_active_samples} muestras")
print(f"Rango de confianzas: {confidences.min():.3f} ")

Probabilidades calculadas para 1000 muestras
Rango de confianzas: 0.734 


In [100]:
# Seleccionar muestras más inciertas
sorted_ixs = np.argsort(confidences)

k_active = 50  # cantidad de muestras inciertas a revisar

# Obtener las muestras más inciertas
X_most_uncertain = X_train[:n_active_samples][sorted_ixs[:k_active]]
y_most_uncertain = y_train[:n_active_samples][sorted_ixs[:k_active]]

print(f"Seleccionadas las {k_active} muestras más inciertas")
print("Las 10 muestras con menor confianza:")
for i in range(min(10, k_active)):
    idx = sorted_ixs[i]
    confidence = confidences[idx]
    true_label = y_train[:n_active_samples][idx]
    letter = chr(ord('A') + true_label) if true_label <= 25 else f"Clase_{true_label}"
    print(f"  Muestra {idx}: Confianza={confidence:.3f} → Letra {letter} (clase {true_label})")

Seleccionadas las 50 muestras más inciertas
Las 10 muestras con menor confianza:
  Muestra 337: Confianza=0.734 → Letra O (clase 14)
  Muestra 480: Confianza=0.751 → Letra M (clase 12)
  Muestra 986: Confianza=0.766 → Letra C (clase 2)
  Muestra 195: Confianza=0.798 → Letra M (clase 12)
  Muestra 176: Confianza=0.802 → Letra D (clase 3)
  Muestra 34: Confianza=0.804 → Letra M (clase 12)
  Muestra 294: Confianza=0.807 → Letra S (clase 18)
  Muestra 984: Confianza=0.807 → Letra C (clase 2)
  Muestra 249: Confianza=0.807 → Letra O (clase 14)
  Muestra 78: Confianza=0.819 → Letra H (clase 7)


In [101]:
# Entrenar con aprendizaje activo
log_reg_active = LogisticRegression(multi_class="ovr", solver="lbfgs", max_iter=5000, random_state=42)

# Entrenar con las muestras más inciertas
log_reg_active.fit(X_most_uncertain, y_most_uncertain)

# Evaluar en el conjunto de prueba
accuracy_active = log_reg_active.score(X_test, y_test)
print(f"Precisión usando aprendizaje activo ({k_active} muestras inciertas): {accuracy_active:.4f}")

Precisión usando aprendizaje activo (50 muestras inciertas): 0.3197
